In [1]:
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.models import Sequential
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random

import warnings
warnings.filterwarnings('ignore')

In [7]:
file_name = 'F01/Session1/wav_arrayMic/0001.wav'
y, sr = librosa.load(file_name, duration=5.0)
ps = librosa.feature.melspectrogram(y=y, sr=sr)
ps.shape

(128, 216)

In [8]:
import os
import numpy as np
from tqdm import tqdm

D = [] # Dataset

directories =[  
                 ['F01','Atypical'],['F03','Atypical'],['F04','Atypical'],['FC01','Typical'],['FC02','Typical'],
                 ['FC03','Typical'],['M01','Atypical'],['M02','Atypical'],['M03','Atypical'],['M04','Atypical'],
                 ['M05','Atypical'],['MC01','Typical'],['MC02','Typical'],['MC03','Typical'],['MC04','Typical']
             ]

for pair in directories:
    directory = pair[0]
    label = pair[1]
    print(directory)
    for filename in os.listdir(directory):
        if filename != 'Notes':
            file = os.path.join(directory, filename)
            file = os.path.join(file, 'wav_arrayMic')
            if os.path.isdir(file):
                for audiofile in tqdm(os.listdir(file)):
                    f = os.path.join(file, audiofile)
                    if os.path.isfile(f):
                        if librosa.get_duration(filename=f) != 0:
                            y, sr = librosa.load(f, duration=5.0)
                            ps = librosa.feature.melspectrogram(y=y, sr=sr)
                            if ps.shape != (128, 216): continue
                            D.append( (ps, label) )

  0%|          | 0/134 [00:00<?, ?it/s]

F01


  0%|          | 1/204 [00:00<00:20,  9.97it/s]

F03


  1%|          | 1/199 [00:00<00:19,  9.97it/s]

...

  0%|          | 1/600 [00:00<01:34,  6.34it/s]

MC03


  0%|          | 1/648 [00:00<01:13,  8.85it/s]

MC04


100%|██████████| 373/373 [00:21<00:00, 17.40it/s]


In [9]:
print("Number of samples: ", len(D))

Number of samples:  1004


In [60]:
dataset = D
random.shuffle(dataset)

train = dataset[:800]
dataset = dataset[800:]
validate = dataset[:150]
test = dataset[150:]

X_train, y_train = zip(*train)
X_validate, y_validate = zip(*validate)
X_test, y_test = zip(*test)

# Reshape for CNN input
X_train = np.array([x.reshape( (128, 216, 1) ) for x in X_train])
X_validate = np.array([x.reshape( (128, 216, 1) ) for x in X_validate])
X_test = np.array([x.reshape( (128, 216, 1) ) for x in X_test])

In [61]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
#y=to_categorical(labelencoder.fit_transform(y))
#y
y_train = np.array(to_categorical(labelencoder.fit_transform(y_train)))
y_validate = np.array(to_categorical(labelencoder.fit_transform(y_validate)))
y_test = np.array(to_categorical(labelencoder.fit_transform(y_test)))

In [62]:
X_train.shape, X_validate.shape, X_test.shape, y_train.shape, y_validate.shape, y_test.shape

((800, 128, 216, 1),
 (150, 128, 216, 1),
 (54, 128, 216, 1),
 (800, 2),
 (150, 2),
 (54, 2))

In [63]:
model = Sequential()
input_shape=(128, 216, 1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 124, 212, 24)      624       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 31, 106, 24)       0         
_________________________________________________________________
activation_20 (Activation)   (None, 31, 106, 24)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 27, 102, 48)       28848     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 51, 48)         0         
_________________________________________________________________
activation_21 (Activation)   (None, 6, 51, 48)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 2, 47, 48)        

In [64]:
model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy'])

model.fit(
	x=X_train, 
	y=y_train,
    epochs=12,
    batch_size=32,
    validation_data= (X_validate, y_validate))

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
25/25 [==============================] - 9s 288ms/step - loss: 7.4256 - accuracy: 0.5440 - val_loss: 1.6896 - val_accuracy: 0.6467
Epoch 2/12
25/25 [==============================] - 7s 280ms/step - loss: 1.6493 - accuracy: 0.6551 - val_loss: 0.4761 - val_accuracy: 0.7000
Epoch 3/12
25/25 [==============================] - 9s 362ms/step - loss: 0.4723 - accuracy: 0.7793 - val_loss: 0.4264 - val_accuracy: 0.8400
Epoch 4/12
25/25 [==============================] - 9s 353ms/step - loss: 0.5380 - accuracy: 0.7746 - val_loss: 0.4139 - val_accuracy: 0.8133
Epoch 5/12
25/25 [==============================] - 9s 363ms/step - loss: 0.4320 - accuracy: 0.8245 - val_loss: 0.3813 - val_accuracy: 0.8600
Epoch 6/12
25/25 [==============================] - 9s 372ms/step - loss: 0.3706 - accuracy: 0.8578 - val_loss: 0.3581 - val_accuracy: 0.8733
Epoch 7/12
25/25 [==============================] - 9s 364ms/step - loss: 0.2719 - accuracy: 0.8882 - val_loss: 0.3331 - val_accuracy: 0.8600
Epoch 

In [67]:
predict = model.predict_classes(X_test)
prediction_class = labelencoder.inverse_transform(predict)
#real_class = labelencoder.inverse_transform(y_test)
#prediction_class

x, y = zip(*test)
y
match = 0
print('  Real       Prediction       Match')
for i in range(0,X_test.shape[0]):
    if(y[i]==prediction_class[i]):
        match+=1
    print(y[i] + '       ' + prediction_class[i] + '       ' + str(y[i]==prediction_class[i]))
print('Matched = ', match)
print('Not Matched = ',(X_test.shape[0] - match))

  Real       Prediction       Match
Atypical       Atypical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Atypical       Typical       False
Typical       Typical       True
Atypical       Atypical       True
Atypical       Atypical       True
Typical       Typical       True
Atypical       Atypical       True
Atypical       Atypical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Atypical       Atypical       True
Atypical       Typical       False
Typical       Typical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Atypical       True
Typical       Atypical       False
Atypical       Typical       False
Typical       Typical       True
Atypical       Atypical       True
Atypical       Atypical       True
Atypical       Typical       False
Typica